In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

import requests
import pandas as pd
import numpy as np
import json
import os
import dotenv
import sys
sys.tracebacklimit = 0 # turn off the error tracebacks

from colorthief import ColorThief
from urllib.request import urlopen
import io

from mvspotifyhelper.mvspotifyhelper import MV

from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay

In [2]:
dotenv.load_dotenv()

username = 'michael_vaden'

spot_id = os.getenv('spot_id')
spot_secret = os.getenv('spot_secret')
redirect_uri= 'https://www.virginia.edu/'

client_credentials_manager = SpotifyClientCredentials(client_id=spot_id, client_secret=spot_secret)

scope = "playlist-modify-public playlist-modify-private playlist-read-private playlist-read-collaborative user-library-modify"

token = util.prompt_for_user_token(username, scope, spot_id, spot_secret, redirect_uri, show_dialog=True)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, auth=token)

In [7]:
stallions = MV.get_tracks_from_url('michael_vaden', 'https://open.spotify.com/playlist/6cGowHo47voxuwdUckhMXx?si=4af76a82e81b4050')

playlist = MV.add_song_features(stallions[['track.uri', 'track.id', 'track.name', 'track.popularity']].dropna()).drop(['id', 'uri', 'track_href', 'analysis_url', 'type'], axis=1)

In [22]:
playlist_for_ML = playlist.iloc[:,3:]

playlist_for_ML[['key', 'mode', 'time_signature']] = playlist_for_ML[['key', 'mode', 'time_signature']].astype('category')

# Zero Padding to account for input size in trained model
playlist_for_ML['new1'] = 0
playlist_for_ML['new2'] = 0

playlist_for_ML.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107 entries, 0 to 106
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   track.popularity  107 non-null    int64   
 1   danceability      107 non-null    float64 
 2   energy            107 non-null    float64 
 3   key               107 non-null    category
 4   loudness          107 non-null    float64 
 5   mode              107 non-null    category
 6   speechiness       107 non-null    float64 
 7   acousticness      107 non-null    float64 
 8   instrumentalness  107 non-null    float64 
 9   liveness          107 non-null    float64 
 10  valence           107 non-null    float64 
 11  tempo             107 non-null    float64 
 12  duration_ms       107 non-null    int64   
 13  time_signature    107 non-null    category
 14  new1              107 non-null    int64   
 15  new2              107 non-null    int64   
dtypes: category(3), float64(9)

In [23]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

num_cols = list(playlist_for_ML.select_dtypes(include=[np.number]))
cat_cols = list(playlist_for_ML.select_dtypes(exclude=[np.number])) # key, mode, time signature

pipeline=ColumnTransformer([
    ('num',StandardScaler(),num_cols),
    ('cat',OneHotEncoder(handle_unknown="ignore"),cat_cols),
])

songs = pipeline.fit_transform(playlist_for_ML)

In [69]:
import joblib

forest_cls = joblib.load('forest1.pkl')

xgb_cls = joblib.load('xgboost1.pkl')

ada_cls = joblib.load('adaboost1.pkl')

nn_cls = joblib.load('neuralnet1.pkl')
#need to fix input size (27 to 29)

In [72]:
labels_forest = forest_cls.predict(songs)
labels_xgb = xgb_cls.predict(songs)
labels_ada = ada_cls.predict(songs)
labels_nn = nn_cls.predict(songs)

4/4 [==============================] - 0s 2ms/step


In [73]:
playlist['weather_type_forest'] = labels_forest
playlist['weather_type_forest'] = playlist['weather_type_forest'].map({0:'rain', 1:'sun'})

playlist['weather_type_xgb'] = labels_xgb
playlist['weather_type_xgb'] = playlist['weather_type_xgb'].map({0:'rain', 1:'sun'})

playlist['weather_type_ada'] = labels_ada
playlist['weather_type_ada'] = playlist['weather_type_ada'].map({0:'rain', 1:'sun'})

playlist['weather_type_nn'] = np.rint(labels_nn)
playlist['weather_type_nn'] = playlist['weather_type_nn'].map({0:'rain', 1:'sun'})

In [75]:
weather_test1 = playlist[['track.name', 'weather_type_forest', 'weather_type_xgb', 'weather_type_ada', 'weather_type_nn']]
weather_test1

,track.name,weather_type_forest,weather_type_xgb,weather_type_ada,weather_type_nn
0,Tieduprightnow,sun,sun,sun,sun
1,Gives You Hell,rain,rain,rain,rain
2,Blossom,sun,sun,sun,sun
3,Stolen Dance,rain,rain,rain,rain
4,The Show Goes On,sun,sun,sun,sun
...,...,...,...,...,...
102,The Spins,rain,rain,rain,rain
103,I Get Around (Mono),rain,rain,rain,rain
104,Modern Jesus,sun,sun,sun,sun
105,Unknown Song,rain,rain,rain,rain
